In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import skimage.io
import keras.backend as K
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout,BatchNormalization ,Activation
from tensorflow.keras.models import Model, Sequential
from keras.applications.nasnet import NASNetLarge
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam

os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
Trial = 'ensemble'

In [ ]:
# train_datagen = ImageDataGenerator(
#     rescale = 1./255,                                  
#     rotation_range=5,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     #zoom_range=0.2,
#     horizontal_flip=True,
#     fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale = 1./255)
# test_datagen  = ImageDataGenerator(rescale = 1./255)

In [ ]:
valid_dataset = valid_datagen.flow_from_directory(directory = '../input/fer2013/test',
                                                  target_size = (224,224),
                                                  interpolation='bilinear',
                                                  class_mode = 'categorical',
                                                  batch_size = 64,
                                                  shuffle=False)

In [ ]:
# test_dataset = test_datagen.flow_from_directory(directory = '../input/fer2013/test',
#                                                   target_size = (48,48),
#                                                   interpolation='bilinear',
#                                                   class_mode = 'categorical',
#                                                   batch_size = 64)

In [ ]:
model1 = tf.keras.applications.ResNet50(input_shape=(224,224,3),include_top=True,weights=None,classes=7)
model1.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model1.load_weights('../input/emotion-util/ResNet_continue.h5')

In [ ]:
model2 = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=True,weights=None,classes=7)
model2.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model2.load_weights('../input/emotion-util/MobileNet.h5')

In [ ]:
model3 = tf.keras.applications.InceptionV3(input_shape=(224,224,3),include_top=True,weights=None,classes=7)
model3.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])
model3.load_weights('../input/emotion-util/InceptionNet.h5')

In [ ]:
pred1 = model1.predict_generator(valid_dataset,max_queue_size=10, workers=1, use_multiprocessing=True, verbose=1)
p1 = pred1.argmax(axis=1)
(p1 == valid_dataset.labels).sum()/7178

In [ ]:
import seaborn as sns
from sklearn import metrics

cm1 = metrics.confusion_matrix(valid_dataset.labels, p1)

plt.figure(figsize=(9,9))
sns.heatmap(cm1, annot=True, fmt=".0f", linewidths=.5, square=True, cmap='Blues')
plt.ylabel('true emotion', size=17)
plt.xlabel('predicted emotion', size=17)
plt.savefig('cm1',dpi=500,bbox_inches = 'tight')

In [ ]:
valid_dataset.labels.shap

In [ ]:
pred2 = model2.predict_generator(valid_dataset,max_queue_size=10, workers=1, use_multiprocessing=True, verbose=1)
p2 = pred2.argmax(axis=1)
(p2 == valid_dataset.labels).sum()/7178

In [ ]:
import seaborn as sns
from sklearn import metrics

cm2 = metrics.confusion_matrix(valid_dataset.labels, p2)

plt.figure(figsize=(9,9))
sns.heatmap(cm2, annot=True, fmt=".0f", linewidths=.5, square=True, cmap='Blues')
plt.ylabel('true emotion', size=17)
plt.xlabel('predicted emotion', size=17)
plt.savefig('cm2',dpi=500,bbox_inches = 'tight')

In [ ]:
pred3 = model3.predict_generator(valid_dataset,max_queue_size=10, workers=1, use_multiprocessing=True, verbose=1)
p3 = pred3.argmax(axis=1)
(p3 == valid_dataset.labels).sum()/7178

In [ ]:
import seaborn as sns
from sklearn import metrics

cm3 = metrics.confusion_matrix(valid_dataset.labels, p3)

plt.figure(figsize=(9,9))
sns.heatmap(cm3, annot=True, fmt=".0f", linewidths=.5, square=True, cmap='Blues')
plt.ylabel('true emotion', size=17)
plt.xlabel('predicted emotion', size=17)
plt.savefig('cm3',dpi=500,bbox_inches = 'tight')

In [ ]:
ensemble_pred = pred1 + pred2 + pred3
ensemble_p = ensemble_pred.argmax(axis=1)
(ensemble_p == valid_dataset.labels).sum()/7178

In [ ]:
import seaborn as sns
from sklearn import metrics

cm4 = metrics.confusion_matrix(valid_dataset.labels, ensemble_p)

plt.figure(figsize=(9,9))
sns.heatmap(cm4, annot=True, fmt=".0f", linewidths=.5, square=True, cmap='Blues')
plt.ylabel('true emotion', size=17)
plt.xlabel('predicted emotion', size=17)
plt.savefig('cm4',dpi=500,bbox_inches = 'tight')